In [8]:
import requests
from getpass import getpass
import json
import jsonlines

In [2]:
session = requests.Session()
session.auth = ('admin', getpass(prompt="Digite sua senha do ElasticSearch"))

Digite sua senha do ElasticSearch········


In [23]:
def search(uri):
    HEADERS = {
        'Content-Type': 'application/json'
    }
    
    query = '''
    {
      "size": 100, 
      "_source": ["texto", "data_publicacao"],
      "sort": [
        {
          "data_publicacao": {
            "order": "desc"
          }
        }
      ]
    }
    '''
    
    
    response = session.get(url=f'{uri}/mapa-fatos/_search', headers=HEADERS, data=query)
    assert response.status_code == 200
    
    return json.loads(response.text)['hits']['hits']

In [33]:
results = search("http://elastic.producao.rancher.tcu.gov.br")
print(len(results))
print(json.dumps(results[0], indent=4, ensure_ascii=False))

100
{
    "_index": "mapa-fatos-2021-01-13-20-33-09",
    "_type": "_doc",
    "_id": "NOTICIA_CLIPPING_53270232",
    "_score": null,
    "_source": {
        "texto": "Em entrevista à Rádio Eldorado, o diretor do Instituto Butantan, Dimas Covas, afirmou que a China deve responder até esta sexta-feira, 14, se enviará a matéria-prima contratada para permitir a retomada da produção da Coronavac no Brasil. Na quarta-feira, 12, o governo paulista se reuniu com autoridades chinesas para alinhar o cronograma. A logística de entrega do IFA está paralisada desde abril, após novos ataques do governo Bolsonaro ao país.  <br>    <br>  O presidente do Butantan afirmou que o último lote de vacinas será entregue nesta sexta-feira: 'São insumos que já deveriam estar aqui em solo brasileiro, por que nesse momento não temos matéria-prima para continuar a produção'.  <br>    <br>  'Nesse momento, não temos autorização do governo da China para importar as vacinas da China. Estamos com intensas negociaçõ

In [47]:
{'id': results[0]['_id'], 'text': results[0]['_source']['texto']}

{'id': 'NOTICIA_CLIPPING_53270232',
 'text': "Em entrevista à Rádio Eldorado, o diretor do Instituto Butantan, Dimas Covas, afirmou que a China deve responder até esta sexta-feira, 14, se enviará a matéria-prima contratada para permitir a retomada da produção da Coronavac no Brasil. Na quarta-feira, 12, o governo paulista se reuniu com autoridades chinesas para alinhar o cronograma. A logística de entrega do IFA está paralisada desde abril, após novos ataques do governo Bolsonaro ao país.  <br>    <br>  O presidente do Butantan afirmou que o último lote de vacinas será entregue nesta sexta-feira: 'São insumos que já deveriam estar aqui em solo brasileiro, por que nesse momento não temos matéria-prima para continuar a produção'.  <br>    <br>  'Nesse momento, não temos autorização do governo da China para importar as vacinas da China. Estamos com intensas negociações com a embaixada aqui no Brasil e com o governo da China através da embaixada brasileira em Pequim.'  <br>    <br>  Segund

In [49]:
with jsonlines.open('mapa_dataset.jsonl', mode='w') as writer:
    for index, result in enumerate(results):
        writer.write({'id': result['_id'], 'text': result['_source']['texto']})